In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import datetime
import time
import os

# Email configurations
sender_email = 'chaitanyakumar521491@gmail.com'
password = os.getenv('EMAIL_PASSWORD')  # Fetch password from environment variable

subject = 'Subject of the email'
message_body = 'Hi, welcome to the world!'

# List of recipient email addresses (populate with actual email addresses)
recipient_emails = [
    'recipient1@example.com',
    'recipient2@example.com',
    # Add up to 1000 recipient email addresses here...
]

# Print the count of recipient emails
print(f'Total number of emails to be sent: {len(recipient_emails)}')

# Function to attach files
def attach_file(msg, file_path):
    filename = os.path.basename(file_path)
    part = MIMEBase('application', 'octet-stream')
    with open(file_path, 'rb') as attachment:
        part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        'Content-Disposition',
        f'attachment; filename= {filename}'
    )
    msg.attach(part)

# Function to send email
def send_email():
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)

        batch_size = 50  # Number of emails to send in one batch
        delay_between_batches = 60  # Delay in seconds between batches

        total_emails = len(recipient_emails)
        total_batches = (total_emails // batch_size) + (1 if total_emails % batch_size > 0 else 0)

        print(f'Total emails to be sent: {total_emails}')
        print(f'Total batches to be sent: {total_batches}')

        for i in range(total_batches):
            start_index = i * batch_size
            end_index = min(start_index + batch_size, total_emails)

            for recipient_email in recipient_emails[start_index:end_index]:
                msg = MIMEMultipart()
                msg['From'] = sender_email
                msg['To'] = recipient_email
                msg['Subject'] = subject

                # Attach body to the email
                msg.attach(MIMEText(message_body, 'plain'))

                # Attach files
                attach_file(msg, r'C:\Users\91949\OneDrive\Desktop\sample\Work Details\pic.jpg')

                # Send the email
                server.sendmail(sender_email, recipient_email, msg.as_string())
                print(f'Email with attachment sent successfully to {recipient_email}')

            print(f'Batch {i+1}/{total_batches} sent successfully. Waiting for next batch...')
            time.sleep(delay_between_batches)

        print('All emails with attachments sent successfully!')

    except Exception as e:
        print(f'Failed to send emails. Error: {e}')

    finally:
        server.quit()

# Specify the target time to send the email (24-hour format)
target_time = datetime.time(14, 0, 0)  # 2:00 PM

# Main loop to check and send email at the specified time
while True:
    current_time = datetime.datetime.now().time()

    if current_time >= target_time:
        send_email()
        break
    
    # Check every 60 seconds
    time.sleep(60)